In [ ]:
import pandas as pd
import ollama
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/dump_14_06_2024_avatar.csv')
df = df.loc[df['user_name'] != 'ths3test']
df = df.loc[df['user_name'] != 'Fest']
df = df.loc[df['user_name'] != 'Konkretes']
df

In [ ]:
def extract_label(response):
    candidate_labels = ["physisch", "psychologisch", "sozial"]

    for label in candidate_labels:
        if label in response:
            return label
    return None

def analyze_sentiment(df, column):

    prompt_template = ("""
    Bitte analysiere den folgenden Text und ordne ihn einer der vorgegebenen Kategorien zu: 'physisch', 'psychologisch', 'sozial'.
    
    Falls der Text keine Informationen enthält, die eine genaue Zuordnung zulassen, antworte mit 'None'.
    
    Es geht darum, aus medizinischem Kontext das Thema zu identifizieren, über das der Text handelt: physische Belange des Körpers, soziale Aktivitäten mit anderen Menschen oder psychologische Themen.
    
    Sobald in dem Text von einer anderen Person berichtet wird, ist es: 'sozial'.
    Sobald in dem Text über körperliche Beschwerden, wie Schmerzen oder körperliche Aktivitäten, gesprochen wird, ist es: 'physisch'.
    Sobald es in dem Text um das psychische Wohlbefinden geht, ist es: 'psychologisch'.
    
    Gib NUR das passende Label zurück, ohne zusätzliche Formatierungen oder Erklärungen.
    
    Text: {}
    """)

    system_message = {
        'role': 'system',
        'content': ("Du bist ein Bewerter, der Texte in Kategorien einteilt. Deine Aufgabe ist es, "
                    "jeden gegebenen Text genau zu analysieren und das passendste Label auszuwählen. "
                    "Es geht darum, das Thema zu identifizieren, über das die Leute sprechen: physische Belange ihres Körpers, soziale Aktivitäten mit anderen Menschen oder psychologische Themen."),
    }

    classifications = []

    for sentence in df[column].fillna(''):
        current_prompt = prompt_template.format(sentence)

        response = ollama.chat(model='llama3', messages=[
            system_message,
            {
                'role': 'user',
                'content': current_prompt,
            },
        ])

        label = extract_label(response['message']['content'])
        classifications.append(label)

    df[column + '_classification'] = classifications

    return df

In [ ]:
# res_yr_activities
res_yr_activities = analyze_sentiment(df, column='yrActivities')
res_selected_res_yr_activities = res_yr_activities[['yrActivities', 'yrActivities_classification']]
res_selected_res_yr_activities

In [ ]:
# res_yr_activities
res_madeYouHappy = analyze_sentiment(df, column='madeYouHappy')
res_selected_res_madeYouHappy = res_madeYouHappy[['madeYouHappy', 'madeYouHappy_classification']]
res_selected_res_madeYouHappy

In [ ]:
# onYrMind
res_onYrMind = analyze_sentiment(df, column='onYrMind')
res_selected_res_onYrMind = res_onYrMind[['onYrMind', 'onYrMind_classification']]
res_selected_res_onYrMind

In [ ]:
# yrDay
res_yrDay = analyze_sentiment(df, column='yrDay')
res_selected_res_yrDay = res_yrDay[['yrDay', 'yrDay_classification']]
res_selected_res_yrDay

In [ ]:
pivot_yr_activities = res_yr_activities.pivot_table(index='user_name', columns='yrActivities_classification', aggfunc='size', fill_value=0)
pivot_madeYouHappy = res_madeYouHappy.pivot_table(index='user_name', columns='madeYouHappy_classification', aggfunc='size', fill_value=0)
pivot_onYrMind = res_onYrMind.pivot_table(index='user_name', columns='onYrMind_classification', aggfunc='size', fill_value=0)
pivot_yrDay = res_yrDay.pivot_table(index='user_name', columns='yrDay_classification', aggfunc='size', fill_value=0)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

pivot_yr_activities.plot(kind='bar', stacked=True, ax=axes[0, 0], legend=False)
axes[0, 0].set_title('yrActivities')
axes[0, 0].set_xlabel('user_name')
axes[0, 0].set_ylabel('Number of Entries')

pivot_madeYouHappy.plot(kind='bar', stacked=True, ax=axes[0, 1], legend=False)
axes[0, 1].set_title('madeYouHappy')
axes[0, 1].set_xlabel('user_name')
axes[0, 1].set_ylabel('Number of Entries')

pivot_onYrMind.plot(kind='bar', stacked=True, ax=axes[1, 0], legend=False)
axes[1, 0].set_title('onYrMind')
axes[1, 0].set_xlabel('user_name')
axes[1, 0].set_ylabel('Number of Entries')

bars = pivot_yrDay.plot(kind='bar', stacked=True, ax=axes[1, 1], legend=False)
axes[1, 1].set_title('yrDay')
axes[1, 1].set_xlabel('user_name')
axes[1, 1].set_ylabel('Number of Entries')

handles, labels = axes[1, 1].get_legend_handles_labels()
fig.legend(handles, labels, title='Categories', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()